## 0. Preparación del notebook e inicialización del cliente de OpenAI API

In [1]:
import os
import pandas as pd
import json
import textwrap
from scipy import spatial
from datetime import datetime
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("../../../../../../../apis/.env")
api_key = os.getenv("OPENAI_API_KEY")
unmasked_chars = 8
masked_key = api_key[:unmasked_chars] + '*' * (len(api_key) - unmasked_chars*2) + api_key[-unmasked_chars:]
print(f"API key: {masked_key}")
client = OpenAI(api_key=api_key)
print("Cliente inicializado como",client)

API key: sk-proj-****************************************************************************************************************************************************-amA_5sA
Cliente inicializado como <openai.OpenAI object at 0x0000021072E3E510>


### 1. Funciones

In [ ]:
def extraer_datos_cv(pre_prompt, schema, cv, model_name="gpt-4o-mini", temperature=0.5):
    """
    Extrae datos estructurados de un CV con OpenAI API.
    Args:
        pre_prompt (str): instrucción para el modelo en lenguaje natural.
        schema (dict): esquema de los parámetros que se espera extraer del CV.
        cv (str): contenido del CV en formato de texto.
        temperature (float, optional): valor de temperatura para el modelo de lenguaje. Por defecto es 0.5.
    Returns:
        pd.DataFrame: DataFrame con los datos estructurados extraídos del CV.
    Raises:
        ValueError: si no se pueden extraer datos estructurados del CV.
    """
    response = client.chat.completions.create(
        model=model_name,
        temperature=temperature,
        messages=[
            {"role": "system", "content": pre_prompt},
            {"role": "user", "content": cv}
        ],
        functions=[
            {
                "name": "extraer_datos_cv",
                "description": "Extrae tabla con títulos de puesto de trabajo, nombres de empresa y períodos de un CV.",
                "parameters": schema
            }
        ],
        function_call="auto"
    )

    if response.choices[0].message.function_call:
        function_call = response.choices[0].message.function_call
        structured_output = json.loads(function_call.arguments)
        if structured_output.get("experiencia"):
            df_cv = pd.DataFrame(structured_output["experiencia"]) 
            return df_cv
        else:
            raise ValueError(f"No se han podido extraer datos estructurados: {response.choices[0].message.content}")
    else:
        raise ValueError(f"No se han podido extraer datos estructurados: {response.choices[0].message.content}")
    

def procesar_periodos(df):    
    """
    Procesa los períodos en un DataFrame y añade columnas con las fechas de inicio, fin y duración en meses. 
    Si no hay fecha de fin, se considera la fecha actual.
    Args:
        df (pandas.DataFrame): DataFrame que contiene una columna 'periodo' con períodos en formato 'YYYYMM-YYYYMM' o 'YYYYMM'.
    Returns:
        pandas.DataFrame: DataFrame con columnas adicionales 'fec_inicio', 'fec_final' y 'duracion'.
            - 'fec_inicio' (datetime.date): Fecha de inicio del período.
            - 'fec_final' (datetime.date): Fecha de fin del período.
            - 'duracion' (int): Duración del período en meses.
    """
    # Función lambda para procesar el período
    def split_periodo(periodo):
        dates = periodo.split('-')
        start_date = datetime.strptime(dates[0], "%Y%m")
        if len(dates) > 1:
            end_date = datetime.strptime(dates[1], "%Y%m")
        else:
            end_date = datetime.now()
        return start_date, end_date

    df[['fec_inicio', 'fec_final']] = df['periodo'].apply(lambda x: pd.Series(split_periodo(x)))

    # Formateamos las fechas para mostrar mes, año, y el primer día del mes (dado que el día es irrelevante y no se suele especificar)
    df['fec_inicio'] = df['fec_inicio'].dt.date
    df['fec_final'] = df['fec_final'].dt.date

    # Añadimos una columna con la duración en meses
    df['duracion'] = df.apply(
        lambda row: (row['fec_final'].year - row['fec_inicio'].year) * 12 + 
                    row['fec_final'].month - row['fec_inicio'].month, 
        axis=1
    )

    return df


def calcula_puntuacion(df, req_experience, positions_cap=4, min_dist_threshold=0.6, max_dist_threshold=0.7):
    """
    Calcula la puntuación de un CV a partir de su tabla de distancias (con respecto a un puesto dado) y duraciones. 

    Params:
    df (pandas.DataFrame): datos de un CV incluyendo diferentes experiencias incluyendo duracies y distancia previamente calculadas sobre los embeddings de un puesto de trabajo
    req_experience (float): experiencia requerida en meses para el puesto de trabajo (valor de referencia para calcular una puntuación entre 0 y 100 en base a diferentes experiencias)
    positions_cap (int, optional): Maximum number of positions to consider for scoring. Defaults to 4.
    min_dist_threshold (float, optional): Distancia entre embeddings a partir de la cual el puesto del CV se considera "equivalente" al de la oferta.
    max_dist_threshold (float, optional): Distancia entre embeddings a partir de la cual el puesto del CV no puntúa.
    
    Returns:
    pandas.DataFrame: DataFrame original añadiendo una columna con las puntuaciones individuales contribuidas por cada puesto.
    float: Puntuación total entre 0 y 100.
    """
    # A efectos de puntuación, computamos para cada puesto como máximo el número total de meses de experiencia requeridos
    df['duration_capped'] = df['duracion'].apply(lambda x: min(x, req_experience))
    # Normalizamos la distancia entre 0 y 1, siendo 0 la distancia mínima y 1 la máxima
    df['adjusted_distance'] = df['distancia'].apply(
        lambda x: 0 if x <= min_dist_threshold else (
            1 if x >= max_dist_threshold else (x - min_dist_threshold) / (max_dist_threshold - min_dist_threshold)
        )
    )
    # Cada puesto puntúa en base a su duración y a la inversa de la distancia (a menor distancia, mayor puntuación)
    df['position_score'] = ((1 - df['adjusted_distance']) * (df['duration_capped']/req_experience) * 100)
    # Descartamos puestos con distancia superior al umbral definido (asignamos puntuación 0), y ordenamos por puntuación
    df.loc[df['distancia'] >= max_dist_threshold, 'position_score'] = 0
    df = df.sort_values(by='position_score', ascending=False)
    # Nos quedamos con los puestos con mayor puntuación (positions_cap)
    df.iloc[positions_cap:, df.columns.get_loc('position_score')] = 0
    # Totalizamos (no debería superar 100 nunca, pero ponemos un límite para asegurar)
    total_score = min(df['position_score'].sum(), 100)
    return df, total_score


def calcula_embeddings_cv(cv_df, column='puesto',model_name="text-embedding-3-small"):
    """
    Calcula los embeddings de una columna de un dataframe con OpenAI API.
    Args:
        cv_df (pandas.DataFrame): DataFrame con los datos de los CV.
        column (str, optional): Nombre de la columna que contiene los datos a convertir en embeddings. Por defecto es 'puesto'.
        model_name (str, optional): Nombre del modelo de embeddings. Por defecto es 'text-embedding-3-small'.
    """
    cv_df['embeddings'] = cv_df[column].apply(
        lambda puesto: client.embeddings.create(
            input=puesto, 
            model=model_name
        ).data[0].embedding
    )
    return cv_df

### 2. Procesamiento de los datos

In [ ]:
# Cargamos el esquema:
with open('../json/ner_schema.json', 'r', encoding='utf-8') as schema_file:
    schema = json.load(schema_file)

# Cargamos el CV:
cv_sample_path = '../../ejemplos_cvs/cv_sample.txt' # Ruta al fichero de texto con un currículo de ejemplo
with open(cv_sample_path, 'r') as file:
    cv_text = file.read()

# Cargamos el prompt para NER:
with open('../prompts/ner_pre_prompt.txt', 'r', encoding='utf-8') as file:
    ner_pre_prompt = file.read()

df_datos_estructurados_cv = extraer_datos_cv(ner_pre_prompt, schema, cv_text)
df_datos_estructurados_cv = procesar_periodos(df_datos_estructurados_cv)

# df_puntuaciones, puntuacion = calcula_puntuacion(df_datos_estructurados_cv,req_experience=12,positions_cap=4,min_dist_threshold=0.55,max_dist_threshold=0.63)
print(f"Puntuación: {puntuacion:.1f}/100")
display(df_puntuaciones)

KeyError: 'distancia'

,empresa,puesto,periodo,fec_inicio,fec_final,duracion
0,Autónomo,Comercial de automoviles,202401-202402,2024-01-01,2024-02-01,1
1,Mercadona,Vendedor/a de puesto de mercado,202310-202403,2023-10-01,2024-03-01,5
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401,2020-01-01,2024-01-01,48
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309,2023-03-01,2023-09-01,6
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305,2020-12-01,2023-05-01,29
5,Bellota Herramientas,Personal de mantenimiento,202005-202011,2020-05-01,2020-11-01,6
